In [53]:
import gzip
import json
import os
import shutil
import pandas as pd
from datetime import datetime
import sqlite3

# Review Existing Unstructured Data and Diagram a New Structured Relational Data Model

Unzip json and read users, brands and receipts

In [54]:
for filename in os.listdir():
    if filename.endswith('.json.gz'):
        try:
            output_file = filename[:-3]  # Remove .gz
            if not os.path.exists(output_file):
                with gzip.open(filename, 'rb') as f_in:
                    with open(output_file, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                print(f"Successfully decompressed {filename}")
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")

In [55]:
def read_jsonl_safe(filename):
    data = []
    with open(filename, 'r') as f:
        for i, line in enumerate(f):
            try:
                data.append(json.loads(line.strip()))
            except json.JSONDecodeError as e:
                print(f"Error on line {i}: {e}")
    return pd.DataFrame(data)

users = read_jsonl_safe('users.json')
receipts = read_jsonl_safe('receipts.json')
brands = read_jsonl_safe('brands.json')

Error on line 0: Expecting value: line 1 column 1 (char 0)
Error on line 495: Expecting value: line 1 column 1 (char 0)


In [56]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
3,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


Extracting values and converting datetime fields

In [57]:
# Define date converter function
def date_converter(x):
    try:
        if isinstance(x, dict) and '$date' in x:
            return pd.to_datetime(x['$date'], unit='ms')
        return pd.NaT  # Return Not a Time for invalid dates
    except:
        return pd.NaT

# Define ObjectId converter function
def id_converter(x):
    try:
        if isinstance(x, dict) and '$oid' in x:
            return x['$oid']
        return str(x)  # Return as string if not in expected format
    except:
        return str(x)

# Apply the conversions
users['_id'] = users['_id'].apply(id_converter)
users['createdDate'] = users['createdDate'].apply(date_converter)
users['lastLogin'] = users['lastLogin'].apply(date_converter)

# Verify the conversions worked
print("\nData types after conversion:")
print(users.dtypes)

# Show sample of converted data
print("\nSample of converted data:")


Data types after conversion:
_id                     object
active                    bool
createdDate     datetime64[ns]
lastLogin       datetime64[ns]
role                    object
signUpSource            object
state                   object
dtype: object

Sample of converted data:


In [58]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
2,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597,consumer,Email,WI
3,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI


In [59]:
# Converting specific fields in receipts
receipts['_id'] = receipts['_id'].apply(id_converter)
receipts['createDate'] = receipts['createDate'].apply(date_converter)
receipts['dateScanned'] = receipts['dateScanned'].apply(date_converter)
receipts['finishedDate'] = receipts['finishedDate'].apply(date_converter)
receipts['modifyDate'] = receipts['modifyDate'].apply(date_converter)
receipts['pointsAwardedDate'] = receipts['pointsAwardedDate'].apply(date_converter)
receipts['purchaseDate'] = receipts['purchaseDate'].apply(date_converter)

# Verify the conversions
print("\nData types after conversion:")
print(receipts.dtypes)

# Show sample of converted data
print("\nSample of converted data:")
print(receipts[['_id', 'createDate', 'dateScanned', 'purchaseDate']].head())

# Check for any null values
print("\nNull value counts:")
print(receipts.isnull().sum())

# Basic statistics for numeric fields
print("\nBasic statistics for numeric fields:")
print(receipts[['pointsEarned', 'bonusPointsEarned', 'purchasedItemCount', 'totalSpent']].describe())


Data types after conversion:
_id                                object
bonusPointsEarned                 float64
bonusPointsEarnedReason            object
createDate                 datetime64[ns]
dateScanned                datetime64[ns]
finishedDate               datetime64[ns]
modifyDate                 datetime64[ns]
pointsAwardedDate          datetime64[ns]
pointsEarned                       object
purchaseDate               datetime64[ns]
purchasedItemCount                float64
rewardsReceiptItemList             object
rewardsReceiptStatus               object
totalSpent                         object
userId                             object
dtype: object

Sample of converted data:
                        _id          createDate         dateScanned  \
0  5ff1e1eb0a720f0523000575 2021-01-03 15:25:31 2021-01-03 15:25:31   
1  5ff1e1bb0a720f052300056b 2021-01-03 15:24:43 2021-01-03 15:24:43   
2  5ff1e1f10a720f052300057a 2021-01-03 15:25:37 2021-01-03 15:25:37   
3  5ff1e1ee0a72

In [60]:
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5,2021-01-03 00:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052


In [61]:
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,False,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,False,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,False,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,False,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,False,TEST BRANDCODE @1612366146827


Explode cpg column as there is more than one element in the dictionary.

In [62]:
brands_norm = pd.json_normalize(brands['cpg'])
brands_norm = brands_norm.add_prefix('cpg.')

In [63]:
brands_final = pd.merge(brands, brands_norm, left_index=True, right_index=True, how='outer')

In [64]:
brands_final['_id'] = brands_final['_id'].apply(id_converter)

In [65]:
brands_final.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,cpg.$ref,cpg.$id.$oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,False,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,False,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,False,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,False,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,False,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7


Connect to SQLite and mount transformed data frames onto DB

In [66]:
import sqlite3
# Connect to SQLite database
conn = sqlite3.connect('database.db')

# Drop duplicates where necessary and save DataFrames to the database
users.drop_duplicates(subset=['_id']).to_sql('users', conn, if_exists='replace', index=False)
brands_final.drop_duplicates(subset=['brandCode']).drop(columns=['cpg'],axis=1).to_sql('brands_final', conn, if_exists='replace', index=False)
receipts.drop(columns=['rewardsReceiptItemList'],axis=1).to_sql('receipts', conn, if_exists='replace', index=False)

# Check tables in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)


           name
0         users
1  brands_final
2      receipts


In [76]:
query = """
SELECT * FROM receipts
"""

result = pd.read_sql_query(query, conn)
result.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,None,2021-01-03 15:25:42,None,5,2021-01-03 00:00:00,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052


# Queries that directly answer predetermined questions from a business stakeholder

When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [80]:
query = """
SELECT rewardsReceiptStatus, Avg(totalSpent)
FROM receipts
GROUP BY 1;
"""

result = pd.read_sql_query(query, conn)
result

,rewardsReceiptStatus,Avg(totalSpent)
0,FINISHED,80.854305
1,FLAGGED,180.451739
2,PENDING,28.032449
3,REJECTED,23.326056
4,SUBMITTED,NaN


When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [82]:
query = """
SELECT rewardsReceiptStatus, SUM(purchasedItemCount) AS totalNoOfItemsPurchased
FROM receipts
GROUP BY 1;
"""

result = pd.read_sql_query(query, conn)
result

,rewardsReceiptStatus,totalNoOfItemsPurchased
0,FINISHED,8184.0
1,FLAGGED,1014.0
2,PENDING,NaN
3,REJECTED,173.0
4,SUBMITTED,NaN


In [85]:
query = """
WITH date_ranges AS (
    SELECT
        MAX(DATE(createDate)) as max_date,
        DATE(MAX(createDate), 'start of month') as current_month,
        DATE(DATE(MAX(createDate), 'start of month'), '-1 month') as prev_month
    FROM receipts
),
current_month_counts AS (
    SELECT brands_final.name AS brand_name,
           COUNT(DISTINCT receipts._id) as current_receipt_count,
           RANK() OVER (ORDER BY COUNT(DISTINCT receipts._id) DESC) as current_rank
    FROM receipts
    JOIN brands_final ON brands_final._id = receipts._id
    WHERE DATE(receipts.createDate) >= (
        SELECT current_month FROM date_ranges
    )
    GROUP BY brands_final.name
),
prev_month_counts AS (
    SELECT brands_final.name AS brand_name,
           COUNT(DISTINCT receipts._id) as prev_receipt_count,
           RANK() OVER (ORDER BY COUNT(DISTINCT receipts._id) DESC) as prev_rank
    FROM receipts
    JOIN brands_final ON brands_final._id = receipts._id
    WHERE DATE(receipts.createDate) >= (SELECT prev_month FROM date_ranges)
    AND DATE(receipts.createDate) < (SELECT current_month FROM date_ranges)
    GROUP BY brands_final.name
)
SELECT
    c.brand_name,
    c.current_receipt_count,
    c.current_rank,
    p.prev_receipt_count,
    p.prev_rank,
    (p.prev_rank - c.current_rank) as rank_change
FROM current_month_counts c
LEFT JOIN prev_month_counts p ON c.brand_name = p.brand_name
WHERE c.current_rank <= 5
ORDER BY c.current_rank;
"""

result = pd.read_sql_query(query, conn)
result

,brand_name,current_receipt_count,current_rank,prev_receipt_count,prev_rank,rank_change


# Evaluate Data Quality Issues in the Data Provided

In [89]:
# Users Data Quality
print("\n=== Users Data Quality Analysis ===")

# Check user creation and login dates
invalid_dates = users[users['lastLogin'] < users['createdDate']]
print(f"\nUsers with lastLogin before createdDate: {len(invalid_dates)}")

# Check state distribution
print("\nState Distribution:")
print(users['state'].value_counts())

# Check role distribution
print("\nRole Distribution:")
print(users['role'].value_counts())

# Check signup source distribution
print("\nSignup Source Distribution:")
print(users['signUpSource'].value_counts())


=== Users Data Quality Analysis ===

Users with lastLogin before createdDate: 0

State Distribution:
state
WI    395
NH     20
AL     12
OH      5
IL      3
KY      1
CO      1
SC      1
Name: count, dtype: int64

Role Distribution:
role
consumer       412
fetch-staff     82
Name: count, dtype: int64

Signup Source Distribution:
signUpSource
Email     442
Google      4
Name: count, dtype: int64


In [92]:
#Receipts Data Quality
print("\n=== Receipts Data Quality Analysis ===")

# Convert receipt dates
date_columns = ['createDate', 'dateScanned', 'finishedDate', 'modifyDate',
                'pointsAwardedDate', 'purchaseDate']
for col in date_columns:
    receipts[col] = receipts[col].apply(date_converter)

# Check numeric values
numeric_columns = ['bonusPointsEarned', 'pointsEarned', 'purchasedItemCount', 'totalSpent']
print("\nNumeric Values Statistics:")
print(receipts[numeric_columns].describe())

# Check receipt status distribution
print("\nReceipt Status Distribution:")
print(receipts['rewardsReceiptStatus'].value_counts())


=== Receipts Data Quality Analysis ===

Numeric Values Statistics:
       bonusPointsEarned  purchasedItemCount
count         544.000000           635.00000
mean          238.893382            14.75748
std           299.091731            61.13424
min             5.000000             0.00000
25%             5.000000             1.00000
50%            45.000000             2.00000
75%           500.000000             5.00000
max           750.000000           689.00000

Receipt Status Distribution:
rewardsReceiptStatus
FINISHED     518
SUBMITTED    434
REJECTED      71
PENDING       50
FLAGGED       46
Name: count, dtype: int64


In [93]:
#Brands Data Quality
print("\n=== Brands Data Quality Analysis ===")

# Check category distribution
print("\nCategory Distribution:")
print(brands['category'].value_counts())

# Check barcode uniqueness
duplicate_barcodes = brands[brands['barcode'].duplicated()]['barcode']
print(f"\nDuplicate barcodes: {len(duplicate_barcodes)}")

# Check brandCode coverage
print("\nBrandCode Missing:")
print(brands['brandCode'].isnull().sum())


=== Brands Data Quality Analysis ===

Category Distribution:
category
Baking                         369
Beer Wine Spirits               90
Snacks                          75
Candy & Sweets                  71
Beverages                       63
Magazines                       44
Health & Wellness               44
Breakfast & Cereal              40
Grocery                         39
Dairy                           33
Condiments & Sauces             27
Frozen                          24
Personal Care                   20
Baby                            18
Canned Goods & Soups            12
Beauty                           9
Cleaning & Home Improvement      6
Deli                             6
Beauty & Personal Care           6
Household                        5
Bread & Bakery                   5
Dairy & Refrigerated             5
Outdoor                          1
Name: count, dtype: int64

Duplicate barcodes: 7

BrandCode Missing:
234


In [96]:
#Relationship Integrity Checks
print("\n=== Relationship Integrity Analysis ===")

# Check user references in receipts
orphan_receipts = receipts[~receipts['userId'].isin(users['_id'])]
print(f"\nReceipts with invalid user references: {len(orphan_receipts)}")



=== Relationship Integrity Analysis ===

Receipts with invalid user references: 148


# Communicate with Stakeholders

Hey team! Quick overview of our data analysis:

Key Findings:

*   Users: Found 494 unique users, but seeing some date inconsistencies (logins before account creation)
*   Receipts: Noticing processing delays and duplicate submissions
*   Brands: Missing barcodes and inconsistent categories in several cases


Quick Questions:

*   What's our policy on duplicate receipts?
*   Is there a max points limit per receipt?
*   Do we expect major user growth soon?


For optimization, additional information needed:



*   Expected data growth rates
*   SLA requirements
*   Backup/recovery requirements
*   Reporting needs
*   Compliance requirements



Production scaling concerns:


*   Peak load handling
*   Database partitioning strategy
*   Caching implementation
*   Batch processing needs
*   Monitoring requirements






